# New Hampshire 2016 — GOP candidate results by county subdivision

#### Import Python tools

In [3]:
# %load_ext lab_black

In [4]:
import pandas as pd
import geopandas as gpd
from pytz import timezone
import urllib.request
import json

In [30]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

#### Timestamps

In [6]:
today_pst = pd.Timestamp("today").strftime(
    "%m/%d/%Y_%H:%M:%S"
)  # Get the current timestamp in Los Angeles time zone

today = pd.Timestamp("today")
eastern_tz = timezone("US/Eastern")
today_est = (
    today.tz_localize("America/Los_Angeles")
    .tz_convert(eastern_tz)
    .strftime("%m/%d/%Y_%H:%M:%S")
)

today_est_timestamp = pd.to_datetime(today_est, format="%m/%d/%Y_%H:%M:%S")
formatted_timestamp = today_est_timestamp.strftime("%-I:%M %p, %b. %d")

---

## Read data

#### Town level results [from NH Secretary of State](https://www.sos.nh.gov/elections/2016-election-results/2016-presidential-primary-election-results/2016-republican)

In [145]:
src = pd.read_excel('data/raw/2016 NH gop results by town.xlsx')
src.columns = src.columns.str.lower().str.replace(' ', '_').str.replace('_write-in', 'write-in')

#### Sum the votes across all candidates for each town

In [146]:
src['town_votes'] = src[['jeb_bush', 'ben_carson', 'chris_christie', 'stephen_comley',
       'tim_cook', 'ted_cruz', 'brooks_cullison', 'matt_drozd', 'daniel_dyas',
       'carly_fiorina', 'jim_gilmore', 'lindsey_graham', 'mike_huckabee',
       'kevin_huey', 'walter_iwachiw', 'bobby_jindal', 'john_kasich',
       'frank_lynch', 'robert_mann', 'andy_martin', 'stephen_mccarthy',
       'peter_messina', 'george_pataki', 'rand_paul', 'chomi_prag',
       'joe_robinson', 'marco_rubio', 'rick_santorum', 'donald_trump',
       'richard_witz', 'write-in']].sum(axis=1)

#### Transform to a long dataframe

In [147]:
src_long = pd.melt(src, id_vars=['town', 'town_votes'], value_vars=['jeb_bush', 'ben_carson', 'chris_christie', 'stephen_comley',
       'tim_cook', 'ted_cruz', 'brooks_cullison', 'matt_drozd', 'daniel_dyas',
       'carly_fiorina', 'jim_gilmore', 'lindsey_graham', 'mike_huckabee',
       'kevin_huey', 'walter_iwachiw', 'bobby_jindal', 'john_kasich',
       'frank_lynch', 'robert_mann', 'andy_martin', 'stephen_mccarthy',
       'peter_messina', 'george_pataki', 'rand_paul', 'chomi_prag',
       'joe_robinson', 'marco_rubio', 'rick_santorum', 'donald_trump',
       'richard_witz', 'write-in'],
        var_name='candidate', value_name='votes').copy()

In [148]:
src_long['vpct'] = ((src_long['votes'] / src_long['town_votes'])*100).round(2)

#### Read a clean list of towns and FIPS codes

In [149]:
towns = pd.read_csv('data/raw/towns.csv', dtype={'co_id':str})

#### Merge results and towns to get clean geographics names, codes

In [166]:
df = pd.merge(src_long, towns, left_on='town', right_on='sec_state_original', how='left', indicator=True)[['co_id', 'town', 'candidate', 'votes', 'vpct', '_merge']].copy()

In [172]:
df[df['_merge'] != 'both']['votes'].sum()

93

#### Clean up canidate names and FIPS column for merging later

In [151]:
candidates = list(df["candidate"].unique())

In [152]:
df["co_id"] = df["co_id"].astype(str)

In [153]:
df["state_abbr"] = "NH"
df["race_type"] = "P"
df["year"] = "2016"
df["pct_reporting"] = 100
df['party'] = 'R'

In [154]:
df = df[
    [
        "candidate",
        "party",
        "co_id",
        "town",
        "state_abbr",
        "race_type",
        "votes",
        "vpct",
        "pct_reporting",
        "year",
    ]
].rename(
    columns={
        # "fips": "county_fips",
        "county_name": "county",
        "vpct": "votes_pct",
    }
)

In [155]:
candidate_votes_grouped = df.groupby(['candidate']).agg({'votes':'sum'}).reset_index().sort_values('votes', ascending=False).reset_index(drop=True)

In [156]:
candidate_votes_grouped

,candidate,votes
0,donald_trump,100704
1,john_kasich,44916
2,ted_cruz,33236
3,jeb_bush,31339
4,marco_rubio,30050
5,chris_christie,21083
6,carly_fiorina,11770
7,ben_carson,6522
8,write-in,2944
9,rand_paul,1930


---

## Geography

#### Import GeoJSON for New Hampshire via Renee

In [13]:
counties_gdf = gpd.read_file(
    "data/raw/NH-towns-elex2022/NH-towns-elex2022.shp"
)
counties_gdf.columns = counties_gdf.columns.str.lower()

---

## Merge

#### First, pivot the canidates table so it's wide

In [14]:
df_wide = (
    df.pivot_table(
        index=["co_id", "county", "year"], columns="candidate", values="votes"
    )
    .reset_index()
    .fillna(0)
)

#### Remove spaces from lowercase column names

In [15]:
df_wide.columns = df_wide.columns.str.lower().str.replace(
    " ", "_"
)

In [16]:
df_wide['other'] = df_wide['_all_others'] + df_wide['_write-in']

In [17]:
df_wide["total16votes"] = df_wide[candidates].sum(axis=1, numeric_only=True)

In [18]:
df_wide["winner"] = (
    df_wide[candidates].idxmax(axis=1).str.replace("_", " ").str.title()
)

In [19]:
df_wide = df_wide.drop(['_all_others', '_write-in'], axis=1)

#### Join results to county geo file

In [20]:
counties_gdf['co_id'] = counties_gdf['co_id'].astype(str)

In [21]:
gdf = gpd.GeoDataFrame(
    pd.merge(df_wide, counties_gdf, left_on="co_id", right_on="co_id")
)

#### Lose the columns we don't need

In [22]:
gdf = gdf.drop(
    [
        'geoid', 'name', 'key', 'state',
       'geoid_real', 'notes', 'co_id'
    ],
    axis=1,
)

---

## Exports

#### Tall county results by candidate and county in CSV format

In [23]:
df.to_csv("data/processed/new_hampshire_2016_gop_results_counties.csv", index=False)

#### Wide county results by candidate and county in CSV format

In [24]:
df_wide.to_csv(
    "data/processed/new_hampshire_2016_gop_results_counties_wide.csv", index=False
)

#### County results in GeoJSON format

In [25]:
gdf.to_file(
    "data/processed/geo/new_hampshire_2016_gop_results_counties.geojson",
    driver="GeoJSON",
)

In [26]:
print(f"The 2016 New Hampshire results exported successfully at {formatted_timestamp}!")

The 2016 New Hampshire results exported successfully at 1:06 PM, Jan. 18!
